In [1]:
import sys
sys.path.append("../../")

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from model.metrics import *
from data.utils import load_img

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

# Choose GPU Device (kidding, we don't need one)

# Set Experiment Name

mse, ssim, featureloss
preprint, gaussian, poisson, svd, downsample

In [4]:
model_1_name = 'featloss-toddler-greyscale'
model_2_name = 'ssim-toddler-greyscale'

data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
dir1 = f'EM/testing/{model_1_name}/real-world_SEM/'  # pssr images
dir2 = f'EM/testing/{model_2_name}/real-world_SEM/'  # our images
dir3 = f'EM/testing/LR-Bilinear/real-world_SEM/'  # bilinear images
targ_dir = f'EM/testing/HR/real-world_SEM/'       # target images

In [5]:
# Modify accordingly
pth1 = data_pth/dir1
pth2 = data_pth/dir2
pth3 = data_pth/dir3
targ_pth = data_pth/targ_dir

dir1_files = sorted(list(pth1.glob('*.tif')))

dir2_files = sorted(list(pth2.glob('*.tif')))

dir3_files = sorted(list(pth3.glob('*.tif')))


targ_files = sorted(list(targ_pth.glob('*.tif')))

print('Processing '+str(len(dir1_files))+' files...')

Processing 42 files...


In [6]:
# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 30
fig_size[1] = 24
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


In [7]:
def visualize_sample(img_fname1, img_fname2, targ_fname, title1=model_1_name, title2="Target", title3=model_2_name):
    pssr_im = load_img(img_fname1) # load image to tensor
    targ_im = load_img(targ_fname)
    our_im = load_img(img_fname2)
    pssr_psnr = np.abs(psnr(torch.tensor(pssr_im), torch.tensor(targ_im)))
    pssr_ssim = np.abs(ssim(torch.tensor(targ_im).unsqueeze(0), torch.tensor(pssr_im).unsqueeze(0)))
    our_psnr = np.abs(psnr(torch.tensor(our_im), torch.tensor(targ_im)))
    our_ssim = np.abs(ssim(torch.tensor(targ_im).unsqueeze(0), torch.tensor(our_im).unsqueeze(0)))
    f, axarr = plt.subplots(1,3) # create visualizations
    axarr[0].imshow(np.squeeze(pssr_im), cmap=plt.cm.gray) # visualize image tensor
    axarr[0].set_title(title1)
    axarr[0].set_xlabel(f"PSNR: {pssr_psnr:.2f}, SSIM: {pssr_ssim:.2f}")
    axarr[1].imshow(np.squeeze(targ_im), cmap=plt.cm.gray) # visualize original image file
    axarr[1].set_title(title2)
    axarr[2].imshow(np.squeeze(our_im), cmap=plt.cm.gray) # visualize image tensor
    axarr[2].set_title(title3)
    axarr[2].set_xlabel(f"PSNR: {our_psnr:.2f}, SSIM: {our_ssim:.2f}")
    plt.show() # show visualization

In [8]:
@interact
def show_sample(sample=33):
    return visualize_sample(dir1_files[sample], dir2_files[sample], targ_files[sample])

interactive(children=(IntSlider(value=33, description='sample', max=99, min=-33), Output()), _dom_classes=('wi…

In [23]:
dir2_files[0]

PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/featureloss-resnet34-pretrained-simplecritic/real-world_SEM/realword_SEM_test_featureloss-resnet34-pretrained-simplecritic.3b_01.tif')

In [7]:
def evaluate_model(pred_files, hr_files):
    psnr_scores = {}
    ssim_scores = {}
    for pred_fname, targ_fname in zip(pred_files, hr_files):
        pred_img = load_img(pred_fname)
        targ_img = load_img(targ_fname)
        
        psnr_score = np.abs(psnr(torch.tensor(pred_img), torch.tensor(targ_img)))
        psnr_scores[targ_fname] = psnr_score
        
        ssim_score = ssim(torch.tensor(targ_img).unsqueeze(0), torch.tensor(pred_img).unsqueeze(0))
        ssim_scores[targ_fname] = ssim_score
    return psnr_scores, ssim_scores

In [8]:
dir2_psnr, dir2_ssim = evaluate_model(dir2_files, targ_files)

In [14]:
dir1_psnr, dir1_ssim = evaluate_model(dir1_files, targ_files)

In [13]:
# DIR1 PSSR PSNR MEAN
np.mean([dir1_psnr[f] for f in dir1_psnr.keys()])

23.384266

In [14]:
# DIR1 PSSR SSIM MEAN
np.mean([dir1_ssim[f] for f in dir1_ssim.keys()])

0.3262943

In [9]:
# MAE PSSR PSNR MEAN
np.mean([dir2_psnr[f] for f in dir2_psnr.keys()])

19.687073

In [10]:
# MAE PSSR SSIM MEAN
np.mean([dir2_ssim[f] for f in dir2_ssim.keys()])

0.24874298

# Simpler looping

In [9]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
dir1 = f'EM/testing/feat_loss_2b/real-world_SEM/'  # pssr images
dir2 = f'EM/testing/feat_loss_imagenet/real-world_SEM/'  # our images
dir3 = f'EM/testing/feat_loss_pssr_baseline/real-world_SEM/'  # bilinear images
targ_dir = f'EM/testing/HR/real-world_SEM/'       # target images

In [10]:
# Modify accordingly
pth1 = data_pth/dir1
pth2 = data_pth/dir2
pth3 = data_pth/dir3
targ_pth = data_pth/targ_dir

dir1_files = sorted(list(pth1.glob('*.tif')))

dir2_files = sorted(list(pth2.glob('*.tif')))

dir3_files = sorted(list(pth3.glob('*.tif')))


targ_files = sorted(list(targ_pth.glob('*.tif')))

print('Processing '+str(len(dir1_files))+' files...')

for model, folder in zip([dir1, dir2, dir3], [dir1_files, dir2_files, dir3_files]):
    dicts = evaluate_model(folder, targ_files)
    scores = []
    for dic in dicts:
        scores.append(np.mean([dic[f] for f in dic.keys()]))
    print(model[11:-16])
    print(f'psnr: {scores[0]}, ssim: {scores[1]}')

Processing 42 files...
feat_loss_pssr_baseline
psnr: 21.097625732421875, ssim: 0.22977863252162933
